In [1]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import PauliList
from qiskit_aer.primitives import Estimator, Sampler

from circuit_knitting.cutting import (
    partition_problem,
    reconstruct_expectation_values,
)

In [2]:
circuit = EfficientSU2(4, entanglement="linear", reps=2).decompose()
circuit.assign_parameters([0.8] * len(circuit.parameters), inplace=True)
observables = PauliList(["ZZII", "IZZI", "IIZZ", "XIXI", "ZIZZ", "IXIX"])

In [3]:
partitioned_problem = partition_problem(
    circuit=circuit,
    partition_labels="AABB",
    num_samples=np.inf,
    observables=observables,
)
partitioned_problem._fields

In [10]:
partitioned_problem.subexperiments.keys()

dict_keys(['A', 'B'])

In [4]:
from circuit_knitting.utils.simulation import ExactSampler
sampler_a = ExactSampler()
sampler_b = ExactSampler()

quasi_dists_a = (
    sampler_a.run(partitioned_problem.subexperiments["A"]).result().quasi_dists
)
quasi_dists_b = (
    sampler_b.run(partitioned_problem.subexperiments["B"]).result().quasi_dists
)

In [5]:
quasi_dists = {"A": quasi_dists_a, "B": quasi_dists_b}
simulated_expvals = reconstruct_expectation_values(
    partitioned_problem,
    quasi_dists,
)

In [6]:
estimator = Estimator(run_options={"shots": None}, approximation=True)
exact_expvals = (
    estimator.run([circuit] * len(observables), list(observables)).result().values
)
print(
    f"Simulated expectation values: {[np.round(simulated_expvals[i], 8) for i in range(len(exact_expvals))]}"
)
print(
    f"Exact expectation values: {[np.round(exact_expvals[i], 8) for i in range(len(exact_expvals))]}"
)
print(
    f"Errors in estimation: {[np.round(simulated_expvals[i]-exact_expvals[i], 8) for i in range(len(exact_expvals))]}"
)
print(
    f"Relative errors in estimation: {[np.round((simulated_expvals[i]-exact_expvals[i]) / exact_expvals[i], 8) for i in range(len(exact_expvals))]}"
)

Simulated expectation values: [0.17153613, 0.1815846, 0.30958691, 0.44036036, 0.08173037, 0.70623815]
Exact expectation values: [0.17153613, 0.1815846, 0.30958691, 0.44036036, 0.08173037, 0.70623815]
Errors in estimation: [-0.0, 0.0, 0.0, -0.0, 0.0, 0.0]
Relative errors in estimation: [-0.0, 0.0, 0.0, -0.0, 0.0, 0.0]
